In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

from PIL import Image

from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("meowmeowmeowmeowmeow/gtsrb-german-traffic-sign")

print("Path to dataset files:", path)

In [2]:
# Extract images and labels from folders

data = []
labels = []
classes = 43

for i in range(classes):
    path = os.path.join(path,'/Train',str(i))
    images = os.listdir(path)

    for a in images:
        try:
            image = Image.open(path + '/'+ a)
            image = image.resize((30,30))
            image = np.array(image) / 255
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

data = np.array(data)
labels = np.array(labels)

In [ ]:
# Data split

X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)

In [ ]:
# Build model

model = Sequential([
    Input(shape=(X_train.shape[1:])),
    Conv2D(filters=32, kernel_size=(5, 5), activation='relu'),
    Conv2D(filters=64, kernel_size=(5, 5), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(rate=0.15),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    Conv2D(filters=256, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(rate=0.20),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(rate=0.25),
    Dense(43, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

In [ ]:
# Train Model

with tf.device('/GPU:0'):
    epochs = 35
    model.fit(X_train, y_train, batch_size=128, epochs=epochs, validation_data=(X_val, y_val))

In [ ]:
# Make predictions on train set

with tf.device('/GPU:0'):
    preds = np.argmax(model.predict(X_train), axis=-1)

y_true = np.argmax(y_train, axis=-1)
acc = accuracy_score(y_true, preds)
f1 = f1_score(y_true, preds, average='weighted')

print(f"Accuracy: {acc}")
print(f"F1 Score: {f1}")

In [8]:
# Extract images from test folder

test_details = pd.read_csv('data/Test.csv')

data = []
labels = test_details["ClassId"].values
imgs = test_details["Path"].values

with tf.device('/GPU:0'):
    for img in imgs:
        image = Image.open('data/' + img)
        image = image.resize([30, 30])
        image = np.array(image) / 255
        data.append(image)

X_test = np.array(data)

In [ ]:
# Make predictions on test set

with tf.device('/GPU:0'):
    preds = np.argmax(model.predict(X_test), axis=-1)

acc = accuracy_score(labels, preds)
f1 = f1_score(labels, preds, average='weighted')

print(f"Accuracy: {acc}")
print(f"F1 Score: {f1}")

In [10]:
# Save model

model.save('traffic_sign_classifier.h5')